#multi GRU

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam,Nadam,RMSprop,SGD
from google.colab import files
from google.colab import drive
import tensorflow as tf

drive.mount('/content/drive')
tf.keras.utils.set_random_seed(2023)

# Load data from CSV file
economic_df=pd.read_csv("drive/My Drive/Master Thesis/lCubicImputation_df.csv")
economic_df= economic_df.apply(lambda x: np.log(x) if np.issubdtype(x.dtype, np.number) else x)
data=economic_df
# Select the input features and the target feature for prediction
#input_features = ['b3']
input_features =['b5']
target_feature = 'b5' # The feature you want to predict

# Prepare data for training
look_back = 3 # Number of previous time steps to consider for prediction-REMAIN 2 CATEGORY FOR 10

X, y = [], []
for i in range(len(data) - look_back):
    X.append(data[input_features].iloc[i:i+look_back].values)
    y.append(data[target_feature].iloc[i+look_back])

X, y = np.array(X), np.array(y)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(1/2), shuffle=False)

# Define hyperparameter values to try

opt_list=[RMSprop,Adam,Nadam,SGD]
units_list = [64,128,256,512,1024]
batch_size_list = [4,8,16,32,64]
epochs_list = [50,100,150]
activation=['linear']

#act='linear'
val_r2=[]
train_r2=[]
test_r2=[]
train_mse=[]
test_mse=[]
hparams=[]
# Loop through hyperparameter combinations
for units in units_list:
    for batch_size in batch_size_list:
        for epochs in epochs_list:
          for opt in opt_list:
            for act in activation:
            # Build the GRU model
              model = Sequential()
              model.add(GRU(units=units, input_shape=(look_back, len(input_features))))
              model.add(Dense(1, activation=act))
              optimizer = opt(learning_rate=0.001)
              model.compile(loss='mean_squared_error', optimizer=optimizer)

              # Train the model with validation set
              history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)

              # Evaluate the model on the validation set
              y_val_pred = model.predict(X_val)
              r2_val = r2_score(y_val, y_val_pred)
              val_r2.append(r2_val)

              # Build the final model with the  hyperparameters
              final_model = Sequential()
              final_model.add(GRU(units=units, input_shape=(look_back, len(input_features))))
              final_model.add(Dense(1, activation=act))
              optimizer = opt(learning_rate=0.001)
              final_model.compile(loss='mean_squared_error', optimizer=optimizer)
              final_model.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)), epochs=epochs, batch_size=batch_size, verbose=0)
              # Evaluate the final model on the test set
              y_test_pred = final_model.predict(X_test)
              mse_test = mean_squared_error(y_test, y_test_pred)
              test_mse.append(mse_test)
              r2_test = r2_score(y_test, y_test_pred)
              test_r2.append(r2_test)
              # Evaluate the final model on the training set
              y_train_pred = final_model.predict(X_train)
              mse_train = mean_squared_error(y_train, y_train_pred)
              train_mse.append(mse_train)
              r2_train = r2_score(y_train, y_train_pred)
              train_r2.append(r2_train)
              hparams.append((look_back,units,batch_size,epochs,act,opt))

result_df=pd.DataFrame({'val_r2':val_r2,'train_mse':train_mse,'test_mse':test_mse,'train_r2':train_r2,'test_r2':test_r2,'hparams':hparams})
result_df

In [ ]:
result_df.to_csv('GRU-B5-multi.csv',index=False)
files.download('GRU-B5-multi.csv')

First Without b1 and b3, look for best multivariate model, then add one of b1 and b3(any one that have bigger corr with out put, or ,best 'only' model found for that)
after all, without using log transform predict real data or normalized of them
only
multi with b1,b3
multi with all
